# RNN Basic

In [1]:
import torch
import torch.nn as nn

In [2]:
batch_size = 40 # 배치 크기
seq_size = 10 # 시퀀스 길이. RNN에서 시퀀스는 순서에 따른 연속된 데이터셋이다.
input_size = 5 # 입력의 크기

hidden_size = 8 # 은닉 상태의 크기

In [3]:
# (batch_size, time_steps = sequence, input_size)
inputs = torch.Tensor(batch_size, seq_size, input_size)

In [4]:
print(inputs[0])

tensor([[-5.6466e-09,  4.5720e-41, -5.6466e-09,  4.5720e-41, -7.6638e+22],
        [ 3.0935e-41, -7.6638e+22,  3.0935e-41,  1.4013e-45,  0.0000e+00],
        [-7.6639e+22,  3.0935e-41, -7.6642e+22,  3.0935e-41,  1.4013e-45],
        [ 0.0000e+00, -7.6639e+22,  3.0935e-41, -7.6639e+22,  3.0935e-41],
        [ 3.3631e-44,  3.0478e+32, -1.9591e-10,  4.5720e-41,  2.8026e-45],
        [ 1.7810e+23,  0.0000e+00,  0.0000e+00,  2.8026e-45,  0.0000e+00],
        [ 2.8026e-45,  8.4078e-45,  3.3631e-44,  3.0263e+29, -5.0409e-21],
        [ 4.5720e-41,  1.4013e-45,  0.0000e+00,  2.8026e-45,  8.4078e-45],
        [ 2.8026e-45,  1.2612e-44,  2.8026e-45,  1.9618e-44,  2.9427e-44],
        [ 4.2131e-41, -7.6639e+22,  3.0935e-41, -5.0405e-21,  4.5720e-41]])


## n_layers = 1

In [5]:
cell = nn.RNN(input_size, hidden_size, batch_first=True)

In [6]:
outputs, _status = cell(inputs) # output은 시간에 따른 최종 레이어의 hidden, status는 레이어별 최종 시간에 따른 hidden
                                # 시간에 따른 최종 출력값을 볼 것이면 outputs, 시간은 최종으로 보고 레이어별로 hidden을 볼 것이면 _status를 쓴다.
                                # 마지막 시간의 마지막 레이어 값을 볼 것이라면 어떤 걸 써도 무방하다.
                                # n_layer = 1인 경우는 항상 마지막 레이어 값을 쓰게 되므로 역시 어느 쪽을 시용해도 된다.

In [7]:
print(outputs.shape) # (batch_size, seq_size, hidden_size) 
                    # output은 sequence(time step)마다의 hidden(output)을 리턴한다. 마지막 값만 쓰기도 하고 전체를 쓰기도 한다. 

torch.Size([40, 10, 8])


In [8]:
print(_status.shape) # (n_layers * n_directions, batch_size, hidden_size)
                    # _status는 RNN 각 레이어, 각 방향별의 현재(최종) hidden을 리턴한다. outputs[:,-1,:]과 같다.

torch.Size([1, 40, 8])


## n_layers = 3 (Deep RNN)

In [9]:
cell = nn.RNN(input_size, hidden_size, num_layers = 3, batch_first=True)

In [10]:
outputs, _status = cell(inputs)

In [11]:
print(outputs.shape) # (batch_size, seq_size, hidden_size)
                    # 최종 레이어의 시퀀스별 hidden(output)을 리턴. 마지막 레이어의 시퀀스별 hidden이 필요하다면 이 값을 사용한다.

torch.Size([40, 10, 8])


In [12]:
print(_status.shape) # (n_layers * n_directions, batch_size, hidden_size)
                    # 각 레이어, 각 방향별 최종 hidden을 리턴. 레이어별 최종 hidden이 필요하다면 이 값을 사용한다.

torch.Size([3, 40, 8])


In [13]:
x = outputs[:, -1, :]

In [14]:
print(x.shape)

torch.Size([40, 8])


## Bidirectional RNN

In [15]:
cell = nn.RNN(input_size, hidden_size, num_layers = 3, batch_first=True, bidirectional=True)

In [16]:
outputs, _status = cell(inputs)

In [17]:
print(outputs.shape) # (batch_size, seq_size, hidden_size * 2)

torch.Size([40, 10, 16])


In [18]:
print(_status.shape) # (n_layers * n_directions, batch_size, hidden_size)

torch.Size([6, 40, 8])


In [19]:
x = outputs[:, -1, :]

In [20]:
print(x.shape)

torch.Size([40, 16])


## batch_first=False

In [21]:
seq_size = 10 # 시퀀스 길이. RNN에서 시퀀스는 순서에 따른 연속된 데이터셋이다.
batch_size = 40 # 배치 크기
input_size = 5 # 입력의 크기

hidden_size = 8 # 은닉 상태의 크기

In [22]:
# batch_first=False인 경우 input/output에서 seq가 제일 먼저 옴. (s, b, i)의 순. batch가 가운데 온다.
# batch 단위로 처리할 것인가, seq 단위로 처리할 것인가의 차이

# (time_steps = sequence, batch_size, input_size)
inputs = torch.Tensor(seq_size, batch_size, input_size)

In [23]:
print(inputs.shape)

torch.Size([10, 40, 5])


In [24]:
cell = nn.RNN(input_size, hidden_size) # batch_first는 디폴트가 False이다

In [25]:
outputs, _status = cell(inputs)

In [26]:
print(outputs.shape) # (seq_size, batch_size, hidden_size)

torch.Size([10, 40, 8])


In [27]:
print(_status.shape) # (n_layers * n_directions, batch_size, hidden_size). hidden은 영향 받지 않는다.

torch.Size([1, 40, 8])
